In [12]:
!pip install python-dotenv



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
from __future__ import annotations
import os
import re
import json
import time
import math
import argparse
import logging
from dataclasses import dataclass, field
from datetime import datetime, timedelta
from typing import Any, Dict, List, Optional, Tuple
from dotenv import load_dotenv

import requests
import pandas as pd
from dateutil import tz

try:
    import yfinance as yf
except ImportError:
    yf = None  # System will degrade gracefully

# Sentiment (degradable): use NLTK VADER if available; otherwise simple heuristics
try:
    from nltk.sentiment import SentimentIntensityAnalyzer
    _HAS_VADER = True
except Exception:
    _HAS_VADER = False

load_dotenv(".env")

True

In [14]:

def fetch_market_timeseries(symbol: str, days: int = 14) -> pd.DataFrame:
    if yf is None:
        print("yfinance not installed; returning empty DataFrame")
        return pd.DataFrame()
    try:
        df = yf.download(symbol, period=f"{days}d", interval="1d", progress=False)
        if not isinstance(df, pd.DataFrame):
            return pd.DataFrame()
        df.reset_index(inplace=True)
        return df
    except Exception as e:
        print(f"yfinance error: {e}")
        return pd.DataFrame()


def fetch_news_newsapi(
    symbol: str, api_key: Optional[str], page_size: int = 10
) -> List[Dict[str, Any]]:
    if not api_key:
        print("NEWSAPI_KEY not set; skipping NewsAPI fetch")
        return []
    url = "https://newsapi.org/v2/everything"
    params = {
        "q": symbol,
        "pageSize": page_size,
        "sortBy": "publishedAt",
        "language": "en",
        "apiKey": api_key,
    }
    try:
        r = requests.get(url, params=params, timeout=20)
        r.raise_for_status()
        data = r.json()
        arts = data.get("articles", [])
        out = []
        for a in arts:
            out.append(
                {
                    "title": a.get("title"),
                    "url": a.get("url"),
                    "publishedAt": a.get("publishedAt"),
                    "source": a.get("source", {}).get("name"),
                    "content": a.get("content") or a.get("description") or "",
                }
            )
        return out
    except Exception as e:
        print(f"NewsAPI error: {e}")
        return []



def fetch_sec_company_facts(symbol: str, app_name: str) -> Dict[str, Any]:
    base = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{symbol}.json" 
    headers = {"User-Agent": app_name}
    try:
        r = requests.get(base, headers=headers, timeout=20)
        if r.status_code == 200:
            return r.json()
    except Exception as e:
        print(f"EDGAR fetch skipped/failed: {e}")
    return {}



def fetch_fred_series(
    series_id: str, api_key: Optional[str], days: int = 365
) -> pd.DataFrame:
    if not api_key:
        print("FRED_API_KEY not set; skipping FRED fetch")
        return pd.DataFrame()
    end = datetime.utcnow().date()
    start = end - timedelta(days=days)
    url = "https://api.stlouisfed.org/fred/series/observations"
    params = {
        "series_id": series_id,
        "api_key": api_key,
        "file_type": "json",
        "observation_start": start.isoformat(),
        "observation_end": end.isoformat(),
    }
    try:
        r = requests.get(url, params=params, timeout=20)
        r.raise_for_status()
        obs = r.json().get("observations", [])
        df = pd.DataFrame(obs)
        if not df.empty:
            df["value"] = pd.to_numeric(df["value"], errors="coerce")
            df["date"] = pd.to_datetime(df["date"])  # NaT for bad
        return df
    except Exception as e:
        print(f"FRED error: {e}")
        return pd.DataFrame()



if __name__ == "__main__":

    # --- Market Data (Yahoo Finance) ---
    print("=== Market Data Test ===")
    df_market = fetch_market_timeseries("AAPL", days=7)
    print(df_market.head())   # show first rows

    # --- News API ---
    print("\n=== News API Test ===")
    news_key = os.getenv("NEWSAPI_KEY")  # make sure this is set in your environment
    news_items = fetch_news_newsapi("AAPL", news_key, page_size=3)
    for n in news_items:
        print(f"- {n['publishedAt']} | {n['title']}")

    # --- SEC EDGAR (⚠️ needs CIK, not ticker) ---
    print("\n=== SEC EDGAR Test ===")
    # Apple’s CIK is 0000320193
    sec_data = fetch_sec_company_facts("0000320193", "AtulPrasad-FDataProject/1.0")
    print(list(sec_data.keys())[:10])   # just show top-level keys

    # --- FRED Macro Series ---
    print("\n=== FRED API Test ===")
    fred_key = os.getenv("FRED_API_KEY")
    df_fred = fetch_fred_series("CPIAUCSL", fred_key, days=90)  # CPI series
    print(df_fred.head())


=== Market Data Test ===
Price        Date       Close        High         Low        Open    Volume
Ticker                   AAPL        AAPL        AAPL        AAPL      AAPL
0      2025-09-11  230.029999  230.449997  226.649994  226.880005  50208600
1      2025-09-12  234.070007  234.509995  229.020004  229.220001  55824200
2      2025-09-15  236.699997  238.190002  235.029999  237.000000  42699500
3      2025-09-16  238.149994  241.220001  236.320007  237.179993  63421100
4      2025-09-17  238.990005  240.100006  237.729996  238.970001  46508000

=== News API Test ===
- 2025-09-18T20:01:30Z | Anker Japan、最大35W出力の巻取り式USB-Cケーブルを一体化した急速充電器「Anker Nano Charger (35W, Built-In 巻取り式 USB-Cケーブル)」のホワイトモデルを発売。
- 2025-09-18T20:01:30Z | Anker Japan、最大35W出力の巻取り式USB-Cケーブルを一体化した急速充電器「Anker Nano Charger (35W, Built-In 巻取り式 USB-Cケーブル)」のホワイトモデルを発売。
- 2025-09-18T19:02:02Z | Nvidia CEO Jensen Huang Says 'You Can’t Overstate the Magic' That is Taiwan Semiconductor, But is TSM Stock a Buy at New Highs?



C:\Users\atul1\AppData\Local\Temp\ipykernel_15192\3998804465.py:72: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end = datetime.utcnow().date()


  realtime_start realtime_end       date    value
0     2025-09-19   2025-09-19 2025-06-01  321.500
1     2025-09-19   2025-09-19 2025-07-01  322.132
2     2025-09-19   2025-09-19 2025-08-01  323.364
